In [1]:
import os
from matplotlib import pyplot as plt
import rasterio
import shapely
import geopandas as gpd
import shutil
import glob
from sklearn.model_selection import train_test_split 
from pyimpute import load_training_vector
from pyimpute import load_targets
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
from sklearn.inspection import partial_dependence
import shap
import pickle
import pandas as pd
from pyimpute import impute
from sklearn import model_selection as mod_sel
from sklearn.inspection import permutation_importance
from sklearn.metrics import confusion_matrix as evaluate_conf
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline

In [2]:
os.chdir("/Users/pranavkulkarni/SDM/Climate_Models_Arenaviruses")
os.getcwd()

'/Users/pranavkulkarni/SDM/Climate_Models_Arenaviruses'

In [3]:
train_xs_list = []
train_y_list = []
test_xs_list = []
test_y_list = []
raster_info = []
target_xs = []

In [4]:
train_xs_list = np.load("/Users/pranavkulkarni/SDM/Climate_Models_Arenaviruses/Data/Inter/train_test_npz/ofl_train_xs.npz")['arr_0']
train_y_list = np.load("/Users/pranavkulkarni/SDM/Climate_Models_Arenaviruses/Data/Inter/train_test_npz/ofl_train_y.npz")['arr_0']
test_xs_list = np.load("/Users/pranavkulkarni/SDM/Climate_Models_Arenaviruses/Data/Inter/train_test_npz/ofl_test_xs.npz")['arr_0']
test_y_list = np.load("/Users/pranavkulkarni/SDM/Climate_Models_Arenaviruses/Data/Inter/train_test_npz/ofl_test_y.npz")['arr_0']
raster_info = np.load("/Users/pranavkulkarni/SDM/Climate_Models_Arenaviruses/Data/Inter/train_test_npz/ofl_raster_info.npz", allow_pickle = True)['arr_0']
target_xs = np.load("/Users/pranavkulkarni/SDM/Climate_Models_Arenaviruses/Data/Inter/train_test_npz/ofl_target_xs.npz")['arr_0']
raster_info = raster_info[0]
raster_info

{'transform': Affine(0.041666666666666664, 0.0, -76.08333333333333,
        0.0, -0.041666666666666664, -17.625),
 'shape': (999, 597),
 'crs': CRS.from_epsg(4326)}

In [5]:
train_xs_list.shape

(50, 256, 24)

In [6]:
feature_names = ["Annual Mean Temperature",
                "Mean Diurnal Range",
                "Isothermality",
                "Temperature Seasonality",
                "Max Temperature of Warmest Month",
                "Min Temperature of Coldest Month",
                "Temperature Annual Range",
                "Mean Temperature of Wettest Quarter",
                "Mean Temperature of Driest Quarter",
                 "Mean Temperature of Warmest Quarter",
                 "Mean Temperature of Coldest Quarter",
                "Annual Precipitation",
                "Precipitation of Wettest Month",
                "Precipitation of Driest Month",
                "Precipitation Seasonality",
                "Precipitation of Wettest Quarter",
                "Precipitation of Driest Quarter",
                 "Precipitation of Warmest Quarter",
                "Precipitation of Coldest Quarter",
                 "LUcrop",
                "LUothr",
                "LUpast",
                "LUsecd",
                "LUurbn"]

In [7]:
ssp2 = sorted(glob.glob("./Data/Input/Processed/Projected/junin/SSP2/*.tif"))
ssp5 = sorted(glob.glob("./Data/Input/Processed/Projected/junin/SSP5/*.tif"))

In [8]:
target_ssp2, ssp2_info = load_targets(ssp2)
target_ssp2[np.isnan(target_ssp2)] = 0

target_ssp5, ssp5_info = load_targets(ssp5)
target_ssp5[np.isnan(target_ssp5)] = 0
print(target_ssp2.shape, target_ssp5.shape)

(596403, 24) (596403, 24)


In [10]:
CLASS_MAP = {
    "rf": (RandomForestClassifier()),
    "et": (ExtraTreesClassifier()),
    "xgb": (XGBClassifier()),
    "lgbm": (LGBMClassifier(verbose = -1))
}

In [11]:
if os.path.exists('./Data/Output/iterations/O_flavescens'):
    print('name folders exists')
else: 
    os.mkdir('./Data/Output/iterations/O_flavescens')
if os.path.exists('./Data/Output/iterations/O_flavescens/current'):
    print('name folders exists')
else: 
    os.mkdir('./Data/Output/iterations/O_flavescens/current')
if os.path.exists('./Data/Output/iterations/O_flavescens/ssp2'):
    print('name folders exists')
else: 
    os.mkdir('./Data/Output/iterations/O_flavescens/ssp2')
if os.path.exists('./Data/Output/iterations/O_flavescens/ssp5'):
    print('name folders exists')
else: 
    os.mkdir('./Data/Output/iterations/O_flavescens/ssp5')

name folders exists
name folders exists
name folders exists
name folders exists


# Single run for test

In [12]:
train_xs = train_xs_list[1]
train_y = train_y_list[1].copy()

test_xs = test_xs_list[1].copy()
test_y = test_y_list[1].copy()
test_xs

array([[2.42360001e+01, 1.17793331e+01, 6.02585068e+01, ...,
        8.05819869e-01, 1.12513510e-06, 9.33267802e-05],
       [8.25533295e+00, 1.84120007e+01, 6.75818558e+01, ...,
        1.05439946e-01, 8.68110538e-01, 0.00000000e+00],
       [2.47531662e+01, 1.21910000e+01, 6.14961624e+01, ...,
        5.07167697e-01, 4.69099097e-02, 0.00000000e+00],
       ...,
       [1.65074997e+01, 1.33616667e+01, 6.01443405e+01, ...,
        3.77655089e-01, 2.91448310e-02, 2.92038335e-03],
       [1.67380009e+01, 1.37046661e+01, 5.21248550e+01, ...,
        3.87691796e-01, 1.93824600e-02, 1.22693996e-03],
       [9.61116695e+00, 1.44569998e+01, 6.00274048e+01, ...,
        4.96651344e-02, 1.42410383e-01, 3.43265274e-04]])

In [15]:
for name, (model) in CLASS_MAP.items():
    rfe = RFE(estimator = model, n_features_to_select = 10)
    pipeline = Pipeline(steps=[('Feature Selection', rfe), ('Model', model)])
    k = 5
    kf = mod_sel.KFold(n_splits = k)
    accu_score = mod_sel.cross_val_score(pipeline, train_xs, train_y, cv = kf, scoring = "accuracy")
    print(name + "CV Accuracy: %0.2f (+/- %0.2f)"
                    % (accu_score.mean() * 100, accu_score.std() * 200))
    accuracy_scores = mod_sel.cross_val_score(pipeline, train_xs, train_y, cv=kf, scoring='roc_auc')
    print(name + "CV AUC_ROC: %0.2f (+/- %0.2f)"
                    % (accuracy_scores.mean() * 100, accuracy_scores.std() * 200))
    accuracy_scores_p = mod_sel.cross_val_score(pipeline, train_xs, train_y, cv=kf, scoring='precision')
    print(name + "CV precision: %0.2f (+/- %0.2f)"
                    % (accuracy_scores_p.mean() * 100, accuracy_scores_p.std() * 200))
    accuracy_scores_r = mod_sel.cross_val_score(pipeline, train_xs, train_y, cv=kf, scoring='recall')
    print(name + "CV recall: %0.2f (+/- %0.2f)"
                    % (accuracy_scores_r.mean() * 100, accuracy_scores_r.std() * 200))
    pipeline.fit(train_xs, train_y)
    y_pred = pipeline.predict(test_xs)
    eval1 = evaluate_conf(test_y, y_pred)
    print(eval1)
    selector = rfe.fit(train_xs, train_y)   
    
    dir_curr = './Data/Output/iterations/O_flavescens/current/' + name + '-images_'
    if os.path.exists(dir_curr):
        print('name folders exists')
    else: 
        os.mkdir(dir_curr)
        
    impute(target_xs, selector, raster_info, outdir = dir_curr, class_prob = True, certainty = True)
        
    dir_ssp2 = './Data/Output/iterations/O_flavescens/ssp2/' + name + '-images_'
    if os.path.exists(dir_ssp2):
        print('name folders exists')
    else: 
        os.mkdir(dir_ssp2)
    impute(target_ssp2, selector, ssp2_info, outdir = dir_ssp2, class_prob = True, certainty = True)
        
    dir_ssp5 = './Data/Output/iterations/O_flavescens/ssp5/' + name + '-images_'
    if os.path.exists(dir_ssp5):
        print('name folders exists')
    else: 
        os.mkdir(dir_ssp5)
    impute(target_ssp5, selector, ssp5_info, outdir = dir_ssp5, class_prob = True, certainty = True)

rfCV Accuracy: 86.33 (+/- 6.57)
rfCV AUC_ROC: 93.76 (+/- 1.35)
rfCV precision: 86.73 (+/- 10.70)
rfCV recall: 84.98 (+/- 4.35)
[[38  5]
 [ 8 35]]
name folders exists
name folders exists
name folders exists
etCV Accuracy: 86.73 (+/- 12.46)
etCV AUC_ROC: 92.79 (+/- 7.63)
etCV precision: 85.73 (+/- 16.61)
etCV recall: 86.89 (+/- 15.27)
[[38  5]
 [ 7 36]]
name folders exists
name folders exists
name folders exists
xgbCV Accuracy: 82.82 (+/- 4.43)
xgbCV AUC_ROC: 92.05 (+/- 6.07)
xgbCV precision: 81.10 (+/- 9.61)
xgbCV recall: 85.01 (+/- 7.95)
[[37  6]
 [ 7 36]]
name folders exists
name folders exists
name folders exists
lgbmCV Accuracy: 84.77 (+/- 4.56)
lgbmCV AUC_ROC: 92.28 (+/- 4.56)
lgbmCV precision: 84.55 (+/- 12.99)
lgbmCV recall: 84.96 (+/- 7.00)
[[40  3]
 [ 8 35]]
name folders exists
name folders exists
name folders exists


In [16]:
sed = selector.fit(train_xs, train_y).support_
np.array(feature_names)[sed]

array(['Temperature Annual Range', 'Mean Temperature of Wettest Quarter',
       'Mean Temperature of Coldest Quarter', 'Annual Precipitation',
       'Precipitation Seasonality', 'Precipitation of Warmest Quarter',
       'LUcrop', 'LUpast', 'LUsecd', 'LUurbn'], dtype='<U35')

# Iterations

## under construction

In [ ]:
%%time

import warnings  
warnings.filterwarnings('ignore')

num_runs = 50

dfs_model = []

list_pd_results = []
list_shap_results = []

features_rf = []
features_et = []
features_xgb = []
features_lgbm = []

for run in range(num_runs):
    model_results = []
    pd_results = []
    shap_results = []
    
    for name, model in CLASS_MAP.items():
        rfe = RFE(estimator = model, n_features_to_select = 10)
        k = 5  
        kf = mod_sel.KFold(n_splits=k)
        selector = rfe.fit(train_xs_list[run], train_y_list[run])
        sed = selector.support_
        accu_score = mod_sel.cross_val_score(selector, 
                                             train_xs_list[run], 
                                             train_y_list[run], 
                                             cv = kf, 
                                             scoring = "accuracy")
        acc_score_mean = accu_score.mean() * 100
        acc_score_std = accu_score.std() * 200
        
        accuracy_scores = mod_sel.cross_val_score(selector, 
                                                          train_xs_list[run], 
                                                          train_y_list[run], 
                                                          cv=kf, scoring='roc_auc')
        roc_auc_mean = accuracy_scores.mean() * 100
        roc_auc_std = accuracy_scores.std() * 200

        accuracy_scores_p = mod_sel.cross_val_score(selector, 
                                                          train_xs_list[run], 
                                                          train_y_list[run], 
                                                          cv=kf, scoring='precision')
        precision_mean = accuracy_scores_p.mean() * 100
        precision_std = accuracy_scores_p.std() * 200

        accuracy_scores_r = mod_sel.cross_val_score(selector, 
                                                          train_xs_list[run], 
                                                          train_y_list[run], 
                                                          cv=kf, scoring='recall')
        recall_mean = accuracy_scores_r.mean() * 100
        recall_std = accuracy_scores_r.std() * 200
        
        y_pred = selector.predict(test_xs_list[run])
        eval1 = evaluate_conf(test_y_list[run], y_pred)
        
        model_results.append({
            'Model': name,
            'Run': run + 1,
            'Acc_mean': acc_score_mean,
            'ACC_std': acc_score_std,
            'ROC_AUC_mean': roc_auc_mean,
            'ROC_AUC_std': roc_auc_std,
            'Precision_mean': precision_mean,
            'Precision_std': precision_std,
            'Recall_mean': recall_mean,
            'Recall_std':recall_std,
            'tp': eval1[0][0],
            'fp': eval1[0][1],
            'fn': eval1[1][0],
            'tn': eval1[1][1]
        })
        
        selector.fit(train_xs_list[run], train_y_list[run])
#         dir_curr = './Data/Output/iterations/C_callosus/current/' + name + '-images_' + str(run)
#         if os.path.exists(dir_curr):
#             print('name folders exists')
#         else: 
#             os.mkdir(dir_curr)
#         impute(target_xs, model, raster_info, outdir = dir_curr, CLAss_prob = True, certainty = True)
        
#         dir_ssp2 = './Data/Output/iterations/C_callosus/ssp2/' + name + '-images_' + str(run)
#         if os.path.exists(dir_ssp2):
#             print('name folders exists')
#         else: 
#             os.mkdir(dir_ssp2)
#         impute(target_ssp2, model, ssp2_info, outdir = dir_ssp2, CLAss_prob = True, certainty = True)
        
#         dir_ssp5 = './Data/Output/iterations/C_callosus/ssp5/' + name + '-images_' + str(run)
#         if os.path.exists(dir_ssp5):
#             print('name folders exists')
#         else: 
#             os.mkdir(dir_ssp5)
#         impute(target_ssp5, model, ssp5_info, outdir = dir_ssp5, CLAss_prob = True, certainty = True)
        for i in np.arange(len(np.array(feature_names)[sed])).tolist():
        
            a_df = pd.DataFrame({np.array(feature_names)[sed][i] + "_grid": 
                  partial_dependence(selector, train_xs, [i])["grid_values"][0], 
                  np.array(feature_names)[sed][i] + "_avg": 
                  partial_dependence(selector, train_xs, [i])["average"][0]})
            pd_results.append(a_df)
            
#         explainer = shap.TreeExplainer(model)  
#         shap_values = explainer.shap_values(train_xs_list[run])
        
#         explainer = shap.TreeExplainer(model)  
#         new_train = train_xs_list[run][:,sed]
#         shap_values = explainer.shap_values(new_train)
#         shap_results.append(shap_values)
        if name == 'rf':
            importances = model.feature_importances_
            forest_importances = pd.Series(importances, index=np.array(feature_names)[sed])
            features_rf.append(forest_importances)
        if name == 'et':
            importances = model.feature_importances_
            et_importances = pd.Series(importances, index=np.array(feature_names)[sed])
            features_et.append(et_importances)        
        if name == 'xgb':
            importances = model.feature_importances_
            xgb_importances = pd.Series(importances, index=np.array(feature_names)[sed])       
            features_xgb.append(xgb_importances)     
        if name == 'lgbm':
            importances = model.feature_importances_
            gain_importance = (model.feature_importances_ / sum(model.feature_importances_)) 
            lgbm_importances = pd.Series(gain_importance, index=np.array(feature_names)[sed]) 
            features_lgbm.append(lgbm_importances)          
    list_pd_results.append(pd_results)
#     list_shap_results.append(shap_results)
    
    model_results_run_df = pd.DataFrame(model_results)
    dfs_model.append(model_results_run_df)
    if (run + 1) % 5 == 0:
        print(f"Completed {run + 1} runs")

Completed 5 runs
Completed 10 runs
Completed 15 runs
Completed 20 runs
Completed 25 runs
Completed 30 runs
Completed 35 runs
Completed 40 runs
Completed 45 runs


In [ ]:
with open('./Data/Output/iterations/O_flavescens/pd_results.pkl', 'wb') as f:
    pickle.dump(list_pd_results, f)
    
# with open('./Data/Output/iterations/O_flavescens/shap_results.pkl', 'wb') as f:
#     pickle.dump(list_shap_results, f)
    
model_results_df = pd.concat(dfs_model, ignore_index=True)
features_rf = pd.concat(features_rf)
features_et = pd.concat(features_et)
features_xgb = pd.concat(features_xgb)
features_lgbm = pd.concat(features_lgbm)

model_results_df.to_csv('./Data/Output/iterations/O_flavescens/model_results.csv', index=True)
features_rf.to_csv('./Data/Output/iterations/O_flavescens/rf_results.csv', index=True)
features_et.to_csv('./Data/Output/iterations/O_flavescens/et_results.csv', index=True)
features_xgb.to_csv('./Data/Output/iterations/O_flavescens/xgb_results.csv', index=True)
features_lgbm.to_csv('./Data/Output/iterations/O_flavescens/lgbm_results.csv', index=True)